In [217]:
# First run src/data_sources/get_game_data.py to get the csv in your working directory.
import pandas as pd
import json
import ast
import numpy as np
import math

In [218]:
# Import csv into dataframe.
raw_games_df = pd.read_csv('games.csv')
raw_games_df.head()

,Unnamed: 0,id,age_ratings,aggregated_rating,aggregated_rating_count,first_release_date,follows,game_modes,genres,involved_companies,name,rating,rating_count,slug,total_rating,version_parent
0,0,3,"[{'id': 3, 'category': 1, 'rating': 11}, {'id'...",83.000000,3.0,1085443200,45.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 13, 'nam...","[{'id': 489, 'company': {'id': 25, 'name': 'Io...",Thief: Deadly Shadows,82.011982,111.0,thief-deadly-shadows,82.505991,NaN
1,1,4,"[{'id': 2930, 'category': 1, 'rating': 11}, {'...",63.642857,17.0,1393286400,96.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 31, 'nam...","[{'id': 6992, 'company': {'id': 27, 'name': 'E...",Thief,70.589275,266.0,thief,67.116066,NaN
2,2,11,"[{'id': 4912, 'category': 1, 'rating': 11}, {'...",72.333333,4.0,1100563200,167.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}]","[{'id': 473, 'company': {'id': 117, 'name': 'T...",Vampire: The Masquerade - Bloodlines,86.532724,499.0,vampire-the-masquerade-bloodlines,79.433029,NaN
3,3,16,"[{'id': 11648, 'category': 1, 'rating': 11}, {...",81.900000,15.0,1287446400,544.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 12, 'nam...","[{'id': 3714, 'company': {'id': 47, 'name': 'O...",Fallout: New Vegas,87.216996,1540.0,fallout-new-vegas,84.558498,NaN
4,4,18,"[{'id': 4498, 'category': 1, 'rating': 11}, {'...",73.800000,3.0,995846400,495.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}]","[{'id': 801, 'company': {'id': 204, 'name': 'G...",Max Payne,85.958331,1401.0,max-payne,79.879166,NaN


In [219]:
raw_games_df.shape

(6349, 16)

In [220]:
# Write function which checks if ESRB rating is 'Mature'.
def is_mature_esrb_rating(str_list_of_dicts):

    # Check for nan values; nan != nan.
    if str_list_of_dicts != str_list_of_dicts:
        return

    # Convert string representation of list of dicts to list of dicts.
    list_of_dicts = ast.literal_eval(str_list_of_dicts)

    # Hard-coded dictionary of ratings from https://api-docs.igdb.com/#age-rating.
    esrb_dict = {
        "6": "RP",
        "7": "EC",
        "8": "E",
        "9": "E10",
        "10": "T",
        "11": "M",
        "12": "AO"
    }

    # Loop through dictionaries in list
    for dictionary in list_of_dicts:

        try:
            # If rating is from ESRB (category 1).
            if dictionary.get('category') == 1:
                
                # Return True if ESRB rating is 'Mature'
                if esrb_dict[str(dictionary.get('rating'))] == 'M':
                    return True

                else:
                    # Return False for all other ratings.
                    return False

        except KeyError:
            print('ESRB rating id not in dictionary.')
            return math.nan

In [221]:
raw_games_df['mature'] = raw_games_df['age_ratings'].map(is_mature_esrb_rating)

In [222]:
print(raw_games_df.shape)
raw_games_df.head()

(6349, 17)


,Unnamed: 0,id,age_ratings,aggregated_rating,aggregated_rating_count,first_release_date,follows,game_modes,genres,involved_companies,name,rating,rating_count,slug,total_rating,version_parent,mature
0,0,3,"[{'id': 3, 'category': 1, 'rating': 11}, {'id'...",83.000000,3.0,1085443200,45.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 13, 'nam...","[{'id': 489, 'company': {'id': 25, 'name': 'Io...",Thief: Deadly Shadows,82.011982,111.0,thief-deadly-shadows,82.505991,NaN,True
1,1,4,"[{'id': 2930, 'category': 1, 'rating': 11}, {'...",63.642857,17.0,1393286400,96.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 31, 'nam...","[{'id': 6992, 'company': {'id': 27, 'name': 'E...",Thief,70.589275,266.0,thief,67.116066,NaN,True
2,2,11,"[{'id': 4912, 'category': 1, 'rating': 11}, {'...",72.333333,4.0,1100563200,167.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}]","[{'id': 473, 'company': {'id': 117, 'name': 'T...",Vampire: The Masquerade - Bloodlines,86.532724,499.0,vampire-the-masquerade-bloodlines,79.433029,NaN,True
3,3,16,"[{'id': 11648, 'category': 1, 'rating': 11}, {...",81.900000,15.0,1287446400,544.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 12, 'nam...","[{'id': 3714, 'company': {'id': 47, 'name': 'O...",Fallout: New Vegas,87.216996,1540.0,fallout-new-vegas,84.558498,NaN,True
4,4,18,"[{'id': 4498, 'category': 1, 'rating': 11}, {'...",73.800000,3.0,995846400,495.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}]","[{'id': 801, 'company': {'id': 204, 'name': 'G...",Max Payne,85.958331,1401.0,max-payne,79.879166,NaN,True


In [223]:
# Write function which checks if ESRB rating is 'Adults Only'.
def is_adults_only_esrb_rating(str_list_of_dicts):

    # Check for nan values; nan != nan.
    if str_list_of_dicts != str_list_of_dicts:
        return

    # Convert string representation of list of dicts to list of dicts.
    list_of_dicts = ast.literal_eval(str_list_of_dicts)

    # Hard-coded dictionary of ratings from https://api-docs.igdb.com/#age-rating.
    esrb_dict = {
        "6": "RP",
        "7": "EC",
        "8": "E",
        "9": "E10",
        "10": "T",
        "11": "M",
        "12": "AO"
    }

    # Loop through dictionaries in list
    for dictionary in list_of_dicts:

        try:
            # If rating is from ESRB (category 1).
            if dictionary.get('category') == 1:
                
                # Return True if ESRB rating is 'Mature'
                if esrb_dict[str(dictionary.get('rating'))] == 'AO':
                    return True

                else:
                    # Return False for all other ratings.
                    return False

        except KeyError:
            print('ESRB rating id not in dictionary.')
            return math.nan

In [224]:
raw_games_df['adults_only'] = raw_games_df['age_ratings'].map(is_adults_only_esrb_rating)

In [225]:
raw_games_df.head()

,Unnamed: 0,id,age_ratings,aggregated_rating,aggregated_rating_count,first_release_date,follows,game_modes,genres,involved_companies,name,rating,rating_count,slug,total_rating,version_parent,mature,adults_only
0,0,3,"[{'id': 3, 'category': 1, 'rating': 11}, {'id'...",83.000000,3.0,1085443200,45.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 13, 'nam...","[{'id': 489, 'company': {'id': 25, 'name': 'Io...",Thief: Deadly Shadows,82.011982,111.0,thief-deadly-shadows,82.505991,NaN,True,False
1,1,4,"[{'id': 2930, 'category': 1, 'rating': 11}, {'...",63.642857,17.0,1393286400,96.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 31, 'nam...","[{'id': 6992, 'company': {'id': 27, 'name': 'E...",Thief,70.589275,266.0,thief,67.116066,NaN,True,False
2,2,11,"[{'id': 4912, 'category': 1, 'rating': 11}, {'...",72.333333,4.0,1100563200,167.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}]","[{'id': 473, 'company': {'id': 117, 'name': 'T...",Vampire: The Masquerade - Bloodlines,86.532724,499.0,vampire-the-masquerade-bloodlines,79.433029,NaN,True,False
3,3,16,"[{'id': 11648, 'category': 1, 'rating': 11}, {...",81.900000,15.0,1287446400,544.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 12, 'nam...","[{'id': 3714, 'company': {'id': 47, 'name': 'O...",Fallout: New Vegas,87.216996,1540.0,fallout-new-vegas,84.558498,NaN,True,False
4,4,18,"[{'id': 4498, 'category': 1, 'rating': 11}, {'...",73.800000,3.0,995846400,495.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}]","[{'id': 801, 'company': {'id': 204, 'name': 'G...",Max Payne,85.958331,1401.0,max-payne,79.879166,NaN,True,False


In [226]:
# Write function which extracts if the game is a single player game.
def is_single_player(str_list_of_dicts):

    # Check for nan values
    if str_list_of_dicts != str_list_of_dicts:
        return

    # Convert string representation of list of dicts to list of dicts.
    list_of_dicts = ast.literal_eval(str_list_of_dicts)

    # Loop through dictionaries in list
    for dict in list_of_dicts:

        # If single player.
        if dict.get('name') == 'Single player':
            return True
                
        else:
            return False

In [227]:
# Write function which extracts if the game is a multiplayer game.
def is_multiplayer(str_list_of_dicts):

    # Check for nan values
    if str_list_of_dicts != str_list_of_dicts:
        return

    # Convert string representation of list of dicts to list of dicts.
    list_of_dicts = ast.literal_eval(str_list_of_dicts)

    # Loop through dictionaries in list
    for dict in list_of_dicts:

        # If multiplayer.
        if dict.get('name') == 'Multiplayer':
            return True
                
        else:
            return False

In [228]:
raw_games_df['single_player'] = raw_games_df['game_modes'].map(is_single_player)

In [229]:
raw_games_df.head()

,Unnamed: 0,id,age_ratings,aggregated_rating,aggregated_rating_count,first_release_date,follows,game_modes,genres,involved_companies,name,rating,rating_count,slug,total_rating,version_parent,mature,adults_only,single_player
0,0,3,"[{'id': 3, 'category': 1, 'rating': 11}, {'id'...",83.000000,3.0,1085443200,45.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 13, 'nam...","[{'id': 489, 'company': {'id': 25, 'name': 'Io...",Thief: Deadly Shadows,82.011982,111.0,thief-deadly-shadows,82.505991,NaN,True,False,True
1,1,4,"[{'id': 2930, 'category': 1, 'rating': 11}, {'...",63.642857,17.0,1393286400,96.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 31, 'nam...","[{'id': 6992, 'company': {'id': 27, 'name': 'E...",Thief,70.589275,266.0,thief,67.116066,NaN,True,False,True
2,2,11,"[{'id': 4912, 'category': 1, 'rating': 11}, {'...",72.333333,4.0,1100563200,167.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}]","[{'id': 473, 'company': {'id': 117, 'name': 'T...",Vampire: The Masquerade - Bloodlines,86.532724,499.0,vampire-the-masquerade-bloodlines,79.433029,NaN,True,False,True
3,3,16,"[{'id': 11648, 'category': 1, 'rating': 11}, {...",81.900000,15.0,1287446400,544.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 12, 'nam...","[{'id': 3714, 'company': {'id': 47, 'name': 'O...",Fallout: New Vegas,87.216996,1540.0,fallout-new-vegas,84.558498,NaN,True,False,True
4,4,18,"[{'id': 4498, 'category': 1, 'rating': 11}, {'...",73.800000,3.0,995846400,495.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}]","[{'id': 801, 'company': {'id': 204, 'name': 'G...",Max Payne,85.958331,1401.0,max-payne,79.879166,NaN,True,False,True


In [230]:
raw_games_df['multiplayer'] = raw_games_df['game_modes'].map(is_multiplayer)

In [231]:
raw_games_df.head()

,Unnamed: 0,id,age_ratings,aggregated_rating,aggregated_rating_count,first_release_date,follows,game_modes,genres,involved_companies,name,rating,rating_count,slug,total_rating,version_parent,mature,adults_only,single_player,multiplayer
0,0,3,"[{'id': 3, 'category': 1, 'rating': 11}, {'id'...",83.000000,3.0,1085443200,45.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 13, 'nam...","[{'id': 489, 'company': {'id': 25, 'name': 'Io...",Thief: Deadly Shadows,82.011982,111.0,thief-deadly-shadows,82.505991,NaN,True,False,True,False
1,1,4,"[{'id': 2930, 'category': 1, 'rating': 11}, {'...",63.642857,17.0,1393286400,96.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 31, 'nam...","[{'id': 6992, 'company': {'id': 27, 'name': 'E...",Thief,70.589275,266.0,thief,67.116066,NaN,True,False,True,False
2,2,11,"[{'id': 4912, 'category': 1, 'rating': 11}, {'...",72.333333,4.0,1100563200,167.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}]","[{'id': 473, 'company': {'id': 117, 'name': 'T...",Vampire: The Masquerade - Bloodlines,86.532724,499.0,vampire-the-masquerade-bloodlines,79.433029,NaN,True,False,True,False
3,3,16,"[{'id': 11648, 'category': 1, 'rating': 11}, {...",81.900000,15.0,1287446400,544.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 12, 'nam...","[{'id': 3714, 'company': {'id': 47, 'name': 'O...",Fallout: New Vegas,87.216996,1540.0,fallout-new-vegas,84.558498,NaN,True,False,True,False
4,4,18,"[{'id': 4498, 'category': 1, 'rating': 11}, {'...",73.800000,3.0,995846400,495.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}]","[{'id': 801, 'company': {'id': 204, 'name': 'G...",Max Payne,85.958331,1401.0,max-payne,79.879166,NaN,True,False,True,False


In [232]:
# Write function which extracts the IGDB developer id.
def get_developer_id(str_list_of_dicts):

    # Check for nan values
    if str_list_of_dicts != str_list_of_dicts:
        return

    # Get json of company information
    with open('companies.json') as json_file:
        companies_dict = json.load(json_file)

    # Get all igdb ids of involved companies.
    igdb_ids_list = []
    for company in companies_dict:
        igdb_ids_list = igdb_ids_list + company['igdb_ids']

    # Convert string representation of list of dicts to list of dicts.
    list_of_dicts = ast.literal_eval(str_list_of_dicts)

    # Create a list of all publisher ids which are in study.
    result = [d['company']['id'] for d in list_of_dicts if (d['company']['id'] in igdb_ids_list and d['developer'] == True)]
    return result



In [233]:
# Write function which extracts the developer id.
def get_publisher_id(str_list_of_dicts):

    # Check for nan values
    if str_list_of_dicts != str_list_of_dicts:
        return

    # Get json of company information
    with open('companies.json') as json_file:
        companies_dict = json.load(json_file)

    # Get all igdb ids of involved companies.
    igdb_ids_list = []
    for company in companies_dict:
        igdb_ids_list = igdb_ids_list + company['igdb_ids']

    # Convert string representation of list of dicts to list of dicts.
    list_of_dicts = ast.literal_eval(str_list_of_dicts)

    # Create a list of all publisher ids which are in study.
    result = [d['company']['id'] for d in list_of_dicts if (d['company']['id'] in igdb_ids_list and d['publisher'] == True)]

    return result


In [234]:
raw_games_df['developer_id'] = raw_games_df['involved_companies'].map(get_developer_id)
raw_games_df['publisher_id'] = raw_games_df['involved_companies'].map(get_publisher_id)

In [235]:
raw_games_df.head()

,Unnamed: 0,id,age_ratings,aggregated_rating,aggregated_rating_count,first_release_date,follows,game_modes,genres,involved_companies,...,rating_count,slug,total_rating,version_parent,mature,adults_only,single_player,multiplayer,developer_id,publisher_id
0,0,3,"[{'id': 3, 'category': 1, 'rating': 11}, {'id'...",83.000000,3.0,1085443200,45.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 13, 'nam...","[{'id': 489, 'company': {'id': 25, 'name': 'Io...",...,111.0,thief-deadly-shadows,82.505991,NaN,True,False,True,False,[],[26]
1,1,4,"[{'id': 2930, 'category': 1, 'rating': 11}, {'...",63.642857,17.0,1393286400,96.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 31, 'nam...","[{'id': 6992, 'company': {'id': 27, 'name': 'E...",...,266.0,thief,67.116066,NaN,True,False,True,False,[],[26]
2,2,11,"[{'id': 4912, 'category': 1, 'rating': 11}, {'...",72.333333,4.0,1100563200,167.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}]","[{'id': 473, 'company': {'id': 117, 'name': 'T...",...,499.0,vampire-the-masquerade-bloodlines,79.433029,NaN,True,False,True,False,[],"[66, 1]"
3,3,16,"[{'id': 11648, 'category': 1, 'rating': 11}, {...",81.900000,15.0,1287446400,544.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 12, 'nam...","[{'id': 3714, 'company': {'id': 47, 'name': 'O...",...,1540.0,fallout-new-vegas,84.558498,NaN,True,False,True,False,[],[248]
4,4,18,"[{'id': 4498, 'category': 1, 'rating': 11}, {'...",73.800000,3.0,995846400,495.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}]","[{'id': 801, 'company': {'id': 204, 'name': 'G...",...,1401.0,max-payne,79.879166,NaN,True,False,True,False,[305],[139]


In [236]:
def get_stock_id_from_igdb_id(id_list):

    # If id_list is empty return np.nan.
    if id_list == False:
        return np.nan

    # Get json of company information
    with open('companies.json') as json_file:
        companies_dict = json.load(json_file)

    # Placeholder values
    stock_id = np.nan
    multiple_id_list = []

    # Loop through igdb_ids in list of developer/publisher ids.
    for igdb_id in id_list:
        # Loop through companies json
        for company in companies_dict:
            # If the igdb_id matches a company from companies json.
            if igdb_id in company['igdb_ids']:
                # Continue if the stock_id is the same as the one already stored.
                if company['stock_id'] == stock_id:
                    continue
                
                # If stock_id not np.nan (placeholder value) and not equal to 
                # stock_id from companies json.
                elif (stock_id != stock_id and (company['stock_id'] != stock_id)):
                    multiple_id_list.append(company['stock_id'])
                    print("Two stock ids for one game " + str(stock_id) + " " + str(company['stock_id']))

                # Set stock_id to the one found in companies json.
                else:
                    stock_id = company['stock_id']

    print(stock_id)
    return stock_id

In [237]:
def get_stock_ids(igdb_ids):

    # Get json of company information
    with open('companies.json') as json_file:
        companies_dict = json.load(json_file)

    stock_ids = []
    for company in companies_dict:
        if any(id in company['igdb_ids'] for id in igdb_ids):
            stock_ids.append(company['stock_id'])
    return stock_ids

In [238]:
raw_games_df['developer_stock_id'] = raw_games_df['developer_id'].map(get_stock_ids)
raw_games_df['publisher_stock_id'] = raw_games_df['publisher_id'].map(get_stock_ids)

In [239]:
raw_games_df.head()

,Unnamed: 0,id,age_ratings,aggregated_rating,aggregated_rating_count,first_release_date,follows,game_modes,genres,involved_companies,...,total_rating,version_parent,mature,adults_only,single_player,multiplayer,developer_id,publisher_id,developer_stock_id,publisher_stock_id
0,0,3,"[{'id': 3, 'category': 1, 'rating': 11}, {'id'...",83.000000,3.0,1085443200,45.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 13, 'nam...","[{'id': 489, 'company': {'id': 25, 'name': 'Io...",...,82.505991,NaN,True,False,True,False,[],[26],[],[JP3164630000]
1,1,4,"[{'id': 2930, 'category': 1, 'rating': 11}, {'...",63.642857,17.0,1393286400,96.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 31, 'nam...","[{'id': 6992, 'company': {'id': 27, 'name': 'E...",...,67.116066,NaN,True,False,True,False,[],[26],[],[JP3164630000]
2,2,11,"[{'id': 4912, 'category': 1, 'rating': 11}, {'...",72.333333,4.0,1100563200,167.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}]","[{'id': 473, 'company': {'id': 117, 'name': 'T...",...,79.433029,NaN,True,False,True,False,[],"[66, 1]",[],"[016721, 18040]"
3,3,16,"[{'id': 11648, 'category': 1, 'rating': 11}, {...",81.900000,15.0,1287446400,544.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 12, 'nam...","[{'id': 3714, 'company': {'id': 47, 'name': 'O...",...,84.558498,NaN,True,False,True,False,[],[248],[],[JP3778630008]
4,4,18,"[{'id': 4498, 'category': 1, 'rating': 11}, {'...",73.800000,3.0,995846400,495.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}]","[{'id': 801, 'company': {'id': 204, 'name': 'G...",...,79.879166,NaN,True,False,True,False,[305],[139],[FI4000251897],[064630]


In [240]:
raw_games_df.shape

(6349, 24)

In [241]:
# Count the number of stock_ids in each column.
raw_games_df['dev_stock_id_count'] = raw_games_df['developer_stock_id'].map(len)
raw_games_df['pub_stock_id_count'] = raw_games_df['publisher_stock_id'].map(len)

In [242]:
raw_games_df.head()

,Unnamed: 0,id,age_ratings,aggregated_rating,aggregated_rating_count,first_release_date,follows,game_modes,genres,involved_companies,...,mature,adults_only,single_player,multiplayer,developer_id,publisher_id,developer_stock_id,publisher_stock_id,dev_stock_id_count,pub_stock_id_count
0,0,3,"[{'id': 3, 'category': 1, 'rating': 11}, {'id'...",83.000000,3.0,1085443200,45.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 13, 'nam...","[{'id': 489, 'company': {'id': 25, 'name': 'Io...",...,True,False,True,False,[],[26],[],[JP3164630000],0,1
1,1,4,"[{'id': 2930, 'category': 1, 'rating': 11}, {'...",63.642857,17.0,1393286400,96.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 31, 'nam...","[{'id': 6992, 'company': {'id': 27, 'name': 'E...",...,True,False,True,False,[],[26],[],[JP3164630000],0,1
2,2,11,"[{'id': 4912, 'category': 1, 'rating': 11}, {'...",72.333333,4.0,1100563200,167.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}]","[{'id': 473, 'company': {'id': 117, 'name': 'T...",...,True,False,True,False,[],"[66, 1]",[],"[016721, 18040]",0,2
3,3,16,"[{'id': 11648, 'category': 1, 'rating': 11}, {...",81.900000,15.0,1287446400,544.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 12, 'nam...","[{'id': 3714, 'company': {'id': 47, 'name': 'O...",...,True,False,True,False,[],[248],[],[JP3778630008],0,1
4,4,18,"[{'id': 4498, 'category': 1, 'rating': 11}, {'...",73.800000,3.0,995846400,495.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}]","[{'id': 801, 'company': {'id': 204, 'name': 'G...",...,True,False,True,False,[305],[139],[FI4000251897],[064630],1,1


In [243]:
raw_games_df['dev_stock_id_count'].value_counts()

dev_stock_id_count
0    4957
1    1389
2       3
Name: count, dtype: int64

In [244]:
raw_games_df[raw_games_df['dev_stock_id_count'] > 1]['name']

1799            Rumble Roses XX
2195     Monster Hunter Stories
2816    New Little King's Story
Name: name, dtype: object

Most games only have one developer stock id. Allow a maximum of two companies per game for simplicity.

In [245]:
raw_games_df['pub_stock_id_count'].value_counts()

pub_stock_id_count
1    5949
0     264
2     135
3       1
Name: count, dtype: int64

Most games only have one publisher stock id. Intend to allow a maximum of two companies per game for simplicity.

In [246]:
# Where there is more than one developer or publisher stock_id, check if they 
# are for the same company.
def check_same_company(stock_ids):

    if stock_ids == []:
        return np.nan

    # Get json of company information
    with open('companies.json') as json_file:
        companies_dict = json.load(json_file)
    
    # Return true
    company_names = []
    for company in companies_dict:
        if company['stock_id'] in stock_ids:
            company_names.append(company['company_name'])

    if all(x == company_names[0] for x in company_names):
        return True
    else:
        return False

In [247]:
raw_games_df['dev_same_company'] = raw_games_df['developer_stock_id'].map(check_same_company)
raw_games_df['pub_same_company'] = raw_games_df['publisher_stock_id'].map(check_same_company)

In [248]:
raw_games_df.head()

,Unnamed: 0,id,age_ratings,aggregated_rating,aggregated_rating_count,first_release_date,follows,game_modes,genres,involved_companies,...,single_player,multiplayer,developer_id,publisher_id,developer_stock_id,publisher_stock_id,dev_stock_id_count,pub_stock_id_count,dev_same_company,pub_same_company
0,0,3,"[{'id': 3, 'category': 1, 'rating': 11}, {'id'...",83.000000,3.0,1085443200,45.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 13, 'nam...","[{'id': 489, 'company': {'id': 25, 'name': 'Io...",...,True,False,[],[26],[],[JP3164630000],0,1,NaN,True
1,1,4,"[{'id': 2930, 'category': 1, 'rating': 11}, {'...",63.642857,17.0,1393286400,96.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 31, 'nam...","[{'id': 6992, 'company': {'id': 27, 'name': 'E...",...,True,False,[],[26],[],[JP3164630000],0,1,NaN,True
2,2,11,"[{'id': 4912, 'category': 1, 'rating': 11}, {'...",72.333333,4.0,1100563200,167.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}]","[{'id': 473, 'company': {'id': 117, 'name': 'T...",...,True,False,[],"[66, 1]",[],"[016721, 18040]",0,2,NaN,False
3,3,16,"[{'id': 11648, 'category': 1, 'rating': 11}, {...",81.900000,15.0,1287446400,544.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 12, 'nam...","[{'id': 3714, 'company': {'id': 47, 'name': 'O...",...,True,False,[],[248],[],[JP3778630008],0,1,NaN,True
4,4,18,"[{'id': 4498, 'category': 1, 'rating': 11}, {'...",73.800000,3.0,995846400,495.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}]","[{'id': 801, 'company': {'id': 204, 'name': 'G...",...,True,False,[305],[139],[FI4000251897],[064630],1,1,True,True


In [249]:
raw_games_df[(raw_games_df['dev_same_company'] == True) & (raw_games_df['dev_stock_id_count'] > 1)]

,Unnamed: 0,id,age_ratings,aggregated_rating,aggregated_rating_count,first_release_date,follows,game_modes,genres,involved_companies,...,single_player,multiplayer,developer_id,publisher_id,developer_stock_id,publisher_stock_id,dev_stock_id_count,pub_stock_id_count,dev_same_company,pub_same_company


There are no games where the same stock_id is used twice for the developer.

In [250]:
raw_games_df[(raw_games_df['dev_same_company'] == True) & (raw_games_df['dev_stock_id_count'] > 1)]

,Unnamed: 0,id,age_ratings,aggregated_rating,aggregated_rating_count,first_release_date,follows,game_modes,genres,involved_companies,...,single_player,multiplayer,developer_id,publisher_id,developer_stock_id,publisher_stock_id,dev_stock_id_count,pub_stock_id_count,dev_same_company,pub_same_company


There are no games where the same stock_id is used twice for the publisher.

In [251]:
# Function for setting first stock_id for each of the two 
# stock_id columns for devs/pubs.
def extract_first_stock_id(list_of_stock_ids):
    
    # Return np.nan if list of stock_ids is empty.
    if len(list_of_stock_ids) < 1:
        return np.nan

    else:
      dev_id = list_of_stock_ids[0]
      return dev_id

In [252]:
# Function for setting second stock_id for each of the two 
# stock_id columns for devs/pubs.
def extract_second_stock_id(list_of_stock_ids):
    
    # Return np.nan if there is only one stock_id in list.
    if len(list_of_stock_ids) < 2:
        return np.nan

    else:
      dev_id = list_of_stock_ids[1]
      return dev_id

In [253]:
raw_games_df['dev1_stock_id'] = raw_games_df['developer_stock_id'].map(extract_first_stock_id)
raw_games_df['dev2_stock_id'] = raw_games_df['developer_stock_id'].map(extract_second_stock_id)
raw_games_df['pub1_stock_id'] = raw_games_df['publisher_stock_id'].map(extract_first_stock_id)
raw_games_df['pub2_stock_id'] = raw_games_df['publisher_stock_id'].map(extract_second_stock_id)

In [254]:
raw_games_df.head()

,Unnamed: 0,id,age_ratings,aggregated_rating,aggregated_rating_count,first_release_date,follows,game_modes,genres,involved_companies,...,developer_stock_id,publisher_stock_id,dev_stock_id_count,pub_stock_id_count,dev_same_company,pub_same_company,dev1_stock_id,dev2_stock_id,pub1_stock_id,pub2_stock_id
0,0,3,"[{'id': 3, 'category': 1, 'rating': 11}, {'id'...",83.000000,3.0,1085443200,45.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 13, 'nam...","[{'id': 489, 'company': {'id': 25, 'name': 'Io...",...,[],[JP3164630000],0,1,NaN,True,NaN,NaN,JP3164630000,NaN
1,1,4,"[{'id': 2930, 'category': 1, 'rating': 11}, {'...",63.642857,17.0,1393286400,96.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 31, 'nam...","[{'id': 6992, 'company': {'id': 27, 'name': 'E...",...,[],[JP3164630000],0,1,NaN,True,NaN,NaN,JP3164630000,NaN
2,2,11,"[{'id': 4912, 'category': 1, 'rating': 11}, {'...",72.333333,4.0,1100563200,167.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 12, 'name': 'Role-playing (RPG)'}]","[{'id': 473, 'company': {'id': 117, 'name': 'T...",...,[],"[016721, 18040]",0,2,NaN,False,NaN,NaN,016721,18040
3,3,16,"[{'id': 11648, 'category': 1, 'rating': 11}, {...",81.900000,15.0,1287446400,544.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}, {'id': 12, 'nam...","[{'id': 3714, 'company': {'id': 47, 'name': 'O...",...,[],[JP3778630008],0,1,NaN,True,NaN,NaN,JP3778630008,NaN
4,4,18,"[{'id': 4498, 'category': 1, 'rating': 11}, {'...",73.800000,3.0,995846400,495.0,"[{'id': 1, 'name': 'Single player'}]","[{'id': 5, 'name': 'Shooter'}]","[{'id': 801, 'company': {'id': 204, 'name': 'G...",...,[FI4000251897],[064630],1,1,True,True,FI4000251897,NaN,064630,NaN


In [255]:
raw_games_df.columns

Index(['Unnamed: 0', 'id', 'age_ratings', 'aggregated_rating',
       'aggregated_rating_count', 'first_release_date', 'follows',
       'game_modes', 'genres', 'involved_companies', 'name', 'rating',
       'rating_count', 'slug', 'total_rating', 'version_parent', 'mature',
       'adults_only', 'single_player', 'multiplayer', 'developer_id',
       'publisher_id', 'developer_stock_id', 'publisher_stock_id',
       'dev_stock_id_count', 'pub_stock_id_count', 'dev_same_company',
       'pub_same_company', 'dev1_stock_id', 'dev2_stock_id', 'pub1_stock_id',
       'pub2_stock_id'],
      dtype='object')

In [256]:
raw_games_df.to_csv('raw_games_flattened.csv')